In [1]:
using LinearAlgebra

### Energy Function to Optimize
$$f(X) = \sum_{i \neq j} \frac{1}{||x_i - x_j||^2}$$
$$||x_i|| = ||x_j|| = 1$$

In [2]:
function potential_energy(X)
    # Compute the potential energy of the particle configuration
    n = size(X,2)
    energy = 0.0
    for i in 1:n-1
        for j in i+1:n
            r = norm(X[:,i] - X[:,j])
            energy += 1 / r^2
        end
    end
    return energy
end

potential_energy (generic function with 1 method)

### Gradient of Energy Function
$$g(X) = \nabla f(X)$$
$$\frac{\partial f}{\partial x_i} = \sum_{j \neq i} \frac{-2(x_i - x_j)}{||x_i - x_j||^4}$$
$$\frac{\partial f}{\partial x_j} = \sum_{i \neq j} \frac{2(x_i - x_j)}{||x_i - x_j||^4}$$
$$||x_i|| = ||x_j|| = 1$$

In [3]:
function gradient(X)
    # Compute the gradient of the potential energy
    k = size(X,1)
    n = size(X,2)
    G = zeros(k, n)
    for i in 1:n-1
        for j in i+1:n
            z = X[:,i] - X[:,j]
            r = norm(z)
            G[:,i] += -2*z / r^4
            G[:,j] += 2*z / r^4
        end
    end
    return G
end

gradient (generic function with 1 method)

### Gradient Descent Algorithm
$$X_{k+1} = X_k - \alpha g(X_k)$$
$$||X_k|| = 1$$

In [4]:
function gradient_descent(X::Matrix, alpha::Float64, tol::Float64, max_iter::Int)
    # X = initial point
    # alpha = step size
    # tol = tolerance for convergence
    # max_iter = maximum number of iterations
    iter = 0
    
    while iter < max_iter
        G = gradient(X)
        X -= alpha .* G
        X = X ./ sqrt.(sum(X.^2, dims=1)) # re-normalize positions
    
        if norm(G) < tol
            print("Converged in ", iter, "iterations")
            break
        end
        
        iter += 1
    end
    
    return X
end

gradient_descent (generic function with 1 method)

### Thompson Problem
Implementation of the Thompson Problem using gradient descent, and a initial random point $X_0$

In [5]:
function random_thomson_problem(k::Int, n::Int, alpha::Float64, tol::Float64, max_iter::Int)
    # Initialize particle positions randomly on the unit sphere
    X = randn(k, n)
    X = X ./ sqrt.(sum(X.^2, dims=1))

    # Gradient Descent
    X = gradient_descent(X, alpha, tol, max_iter)
    
    return X
end

random_thomson_problem (generic function with 1 method)

### Some Results
$k=3, n=2$ to test the functions. That is, a 3D Sphere with just 2 particles.

In [6]:
X = random_thomson_problem(3,2, 0.01, 1e-6, 10000)
fx = potential_energy(X)
gx = gradient(X)
print("Energy: ", fx, "\nGrad: ", norm(gx), "\nX1 Norm: ", norm(X[1:3]), "\nX2 Norm: ", norm(X[4:6]))
X

Energy: 0.5
Grad: 1.4142135623730951
X1 Norm: 1.0
X2 Norm: 1.0

3×2 Matrix{Float64}:
 0.348187  -0.348187
 0.575102  -0.575102
 0.740286  -0.740286

### Initial point is already a solution

In [7]:
X = [0.0 0.0; 1 -1; 0.0 0.0]
fx = potential_energy(X)
gx = gradient(X)
print("Energy: ", fx, "\nGrad: ", norm(gx), "\nX1 Norm: ", norm(X[1:3]), "\nX2 Norm: ", norm(X[4:6]))
X

Energy: 0.5
Grad: 1.4142135623730951
X1 Norm: 1.0
X2 Norm: 1.0

3×2 Matrix{Float64}:
 0.0   0.0
 1.0  -1.0
 0.0   0.0